In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
data_dir = "/home/aarav/1_pytor_train/data/horse-or-human"

In [ ]:
valid_size = .2

In [ ]:
# Normalize the test and train sets with torchvision
train_transforms = transforms.Compose([transforms.Resize(224),
                                           transforms.ToTensor(),
                                           ])

test_transforms = transforms.Compose([transforms.Resize(224),
                                          transforms.ToTensor(),
                                          ])


In [ ]:
# ImageFolder class to load the train and test images
train_data = datasets.ImageFolder(data_dir, transform=train_transforms)
test_data = datasets.ImageFolder(data_dir, transform=test_transforms)

In [ ]:
# Number of train images
num_train = len(train_data)
indices = list(range(num_train))

In [ ]:
train_data.classes

In [ ]:
# Split = 20% of train images
split = int(np.floor(valid_size * num_train))
# Shuffle indices of train images
np.random.shuffle(indices)

In [ ]:
split

In [ ]:
num_train

In [ ]:
# Subset indices for test and train
train_idx, test_idx = indices[split:], indices[:split]

In [ ]:
len(train_idx)
len(test_idx)

In [ ]:
# Samples elements randomly from a given list of indices
train_sampler = SubsetRandomSampler(train_idx)
test_sampler = SubsetRandomSampler(test_idx)

In [ ]:
# Batch and load the images
trainloader = torch.utils.data.DataLoader(train_data, sampler=train_sampler, batch_size=10)
testloader = torch.utils.data.DataLoader(test_data, sampler=test_sampler, batch_size=10)

In [ ]:
print(trainloader.dataset.classes)

In [ ]:
examples = iter(trainloader)
samples ,labels = examples.next()
samples.shape

In [ ]:
def imshow(img,title):
    #img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.figure(figsize=(25,20))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.title(title)
    plt.show()

In [ ]:
out = torchvision.utils.make_grid(samples)

In [ ]:
class_name = train_data.classes

In [ ]:
imshow(out,title=[class_name[x] for x in labels])

In [ ]:
def imshow(img):
    #img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.figure(figsize=(8,8))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    #plt.title(title)
    plt.show()
imshow(samples[0])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = models.resnet18(pretrained=True)

In [ ]:
#freeze all params
for params in model.parameters():
  params.requires_grad_ = False

In [ ]:
#add a new final layer
nr_filters = model.fc.in_features  #number of input features of last layer
model.fc = nn.Sequential(nn.Linear(nr_filters,512),
                        nn.ReLU(),
                        nn.Dropout(0.5),
                        nn.Linear(512,2))

In [ ]:
model

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
model.to(device)

In [ ]:
#Train the network
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        #labels = labels.unsqueeze(1).float()
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if (epoch+1) % 1 == 0:    # 
            print(f'epoch:{epoch+1},loss={running_loss}')
            running_loss = 0.0

print('Finished Training')

In [ ]:
test = samples[4].view(1,3,224,224)

In [ ]:
test = test.to(device)

In [ ]:
model(test)

In [ ]:
def visualize_model(model, num_images=10):
    images_so_far = 0
    fig = plt.figure(figsize=(15, 10))
    
    for i, data in enumerate(testloader):
        inputs, labels = data
        #if use_cuda:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        #print(outputs.round())
        preds = torch.max(outputs,dim=1)[1]
        preds = preds.cpu()
        preds = preds.numpy()
        #print(preds)
        #print(preds[1].item())
        #print(int(preds[0]))
        for j in range(inputs.size()[0]):
            images_so_far += 1
            ax = plt.subplot(2, num_images//2, images_so_far)
            ax.axis('off')
            ax.set_title('predictes: {}'.format(test_data.classes[int(preds[j])]))
            #print(inputs[0].shape)
            mpimg = inputs[j].cpu()
            imshow(mpimg)
            
            if images_so_far == num_images:
                return 

plt.ion()
visualize_model(model)
plt.ioff()